In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:

# Load datasets
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')

In [3]:
# Merge datasets on movieId
data = pd.merge(ratings, movies, on="movieId")

In [4]:
# Create a user-item matrix (pivot table)
user_movie_matrix = data.pivot_table(index='userId', columns='title', values='rating')

In [5]:
# Fill missing values with 0 (for similarity calculations)
user_movie_matrix.fillna(0, inplace=True)

In [6]:
# Compute cosine similarity between users
user_similarity = cosine_similarity(user_movie_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_matrix.index, columns=user_movie_matrix.index)

In [13]:
# Function to get movie recommendations for a given user
def recommend_movies(user_id, num_recommendations=5):
    if user_id not in user_similarity_df.index:
        return "User ID not found."

    # Find similar users
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]

    recommendations = pd.Series(dtype='float64')

    # Get movies rated by similar users
    for similar_user in similar_users:
        rated_movies = user_movie_matrix.loc[similar_user]
        rated_movies = rated_movies[rated_movies > 0]

        # Use pd.concat() instead of append()
        recommendations = pd.concat([recommendations, rated_movies])

    # Aggregate scores and sort
    recommendations = recommendations.groupby(recommendations.index).mean()
    recommended_movies = recommendations.sort_values(ascending=False).head(num_recommendations)

    return recommended_movies


In [14]:
# Example usage
user_id = 1  # Change this based on available user IDs
print("Recommended Movies for User", user_id)
print(recommend_movies(user_id))

Recommended Movies for User 1
Kika (1993)                                                                                                        5.0
21 Up (1977)                                                                                                       5.0
Enigma of Kaspar Hauser, The (a.k.a. Mystery of Kaspar Hauser, The) (Jeder für sich und Gott Gegen Alle) (1974)    5.0
Barenaked in America (1999)                                                                                        5.0
Mothra (Mosura) (1961)                                                                                             5.0
dtype: float64


In [15]:
# Example usage
user_id = 8 # Change this based on available user IDs
print("Recommended Movies for User", user_id)
print(recommend_movies(user_id))

Recommended Movies for User 8
Rocky VI (1986)                                            5.0
Saddest Music in the World, The (2003)                     5.0
Saga of Gosta Berling, The (Gösta Berlings saga) (1924)    5.0
Safe Haven (2013)                                          5.0
More (1998)                                                5.0
dtype: float64
